# Modelo de recomendação de filmes com base em conteúdo

O objetivo deste script é desenvolver um modelo que possibilite a recomendação de filmes com base em conteúdo.
* Principais dependências utilizadas: dask com sklearn.
* Datasets usados: tags e movies extraídos do Movielens.
* Algoritmo selecionado: Similaridade do Cosseno (com TF-IDF)

In [5]:
import os
from dask import dataframe as dd
import dask.array as da
import dask.bag as db
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, linear_kernel
from pymongo import MongoClient, UpdateOne
from itertools import chain
import pandas as pd
import numpy as np
import time
import csv
pd.options.display.precision = 2
pd.options.display.max_rows = 10
normalize_unicode = "NFKD"

## Carregar os dados referentes às tags.

O código abaixo carrega as colunas **tag** e **movieId** do dataset de tags. Além disso, exibe o total de ocorrências de tagueamentos no dataset.

In [6]:
tags_df = dd.read_csv('tags.csv', usecols=["movieId", "tag"])
tags_df.compute()


,movieId,tag
0,260,classic
1,260,sci-fi
2,1732,dark comedy
3,1732,great dialogue
4,7569,so bad it's good
...,...,...
1093355,66934,Neil Patrick Harris
1093356,103341,cornetto trilogy
1093357,189169,comedy
1093358,189169,disabled


## Tratamento das Tags
O trecho a seguir executa o tratamento dos dados presentes em tags a fim de usá-las para enriquecer o dataset de filmes no que se refere à recomendação com base em tags. Além disso, remove as tags repetidas.

In [7]:
tags_df["tag"] = tags_df["tag"].str.lower()
tags_df["tag"] = tags_df["tag"].str.normalize(normalize_unicode)
tags = tags_df.drop_duplicates()
tags_array = tags.groupby('movieId')['tag']\
.apply(list)\
.to_frame()
tags_array.compute()

/Users/jeffersonsantos/movies_recommendation_tcc/.venv/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  """


,tag
movieId,
1,"[owned, imdb top 250, pixar, time travel, chil..."
2,"[robin williams, time travel, fantasy, based o..."
3,"[funny, best friend, duringcreditsstinger, fis..."
4,"[based on novel or book, chick flick, divorce,..."
5,"[aging, baby, confidence, contraception, daugh..."
...,...
208813,[might like]
208933,"[black and white, deal with the devil]"
209035,"[computer animation, japan, mass behavior, mas..."


## Gerar um novo arquivo de tags
Este arquivo contém todas as tags em minúsculo e sem os valores repetidos. No entanto, erros de digitação serão computados como tags distintas.

In [8]:
path = os.path.abspath(os.getcwd())
tags_df.to_csv(f"{path}/tags_lower.csv", single_file=True)

['/Users/jeffersonsantos/movies_recommendation_tcc/tags_lower.csv']

## Construção dos dados de filmes
O trecho de código exibido a seguir realiza o load dos metadados do dataset **movies**. Em posse desses dados, é feita a unificação com as **tags** para oferecer mais insumos ao processo de recomendação.

In [9]:
movies_df = dd.read_csv('movies.csv')
movies_df["genres"] = movies_df["genres"].str.lower()
movies_df["genres"] = movies_df["genres"].str.normalize(normalize_unicode)
movies_and_tags_df = movies_df.merge(tags_array, on='movieId', how='left')
movies_and_tags_df["tag"]=movies_and_tags_df["tag"].str.join("|")
movies_and_tags_df.compute()

,movieId,title,genres,tag
0,1,Toy Story (1995),adventure|animation|children|comedy|fantasy,owned|imdb top 250|pixar|time travel|children|...
1,2,Jumanji (1995),adventure|children|fantasy,robin williams|time travel|fantasy|based on ch...
2,3,Grumpier Old Men (1995),comedy|romance,funny|best friend|duringcreditsstinger|fishing...
3,4,Waiting to Exhale (1995),comedy|drama|romance,based on novel or book|chick flick|divorce|int...
4,5,Father of the Bride Part II (1995),comedy,aging|baby|confidence|contraception|daughter|g...
...,...,...,...,...
62418,209157,We (2018),drama,NaN
62419,209159,Window of the Soul (2001),documentary,NaN
62420,209163,Bad Poems (2018),comedy|drama,NaN
62421,209169,A Girl Thing (2001),(no genres listed),NaN


## Filmes com nomes repetidos
Ao observar o dataset de filmes com mais atenção, perecebeu-se a presença de títulos de filmes repetidos. Apesar de conter o nome e o ano de lançamento, este não é um identificador forte o bastante para ser usado como balizador de que se tratam de filmes distintos. Por conta disso, optou-se por levar em consideração os ids diferentes e tratá-los como obras diferenciadas.

In [10]:
movie_data = movies_and_tags_df.compute()
movie_data["title"][movie_data["title"].isin(movie_data["title"][movie_data["title"].duplicated()])]

580             Aladdin (1992)
1710      Men with Guns (1997)
2553            Dracula (1931)
2759           Saturn 3 (1980)
3454             Gossip (2000)
                 ...          
61525      Lost & Found (2018)
61697            Camino (2016)
61714    American Woman (2019)
61800        The Plague (2006)
61913    American Woman (2019)
Name: title, Length: 196, dtype: object

### JUSTIFICATIVA PARA O USO DO JOIN DE FILMES COM TAGS
Cerca de 72% dos filmes da base possuem tags. Por conta disso, optou-se por utilizar este campo juntamente com gêneros. O trecho abaixo ajuda a referendar esta assertiva.

In [11]:
movies_without_tags = movies_and_tags_df[(movies_and_tags_df['tag'].isna())]
percentage_of_movies_with_tags = 100 - (movies_without_tags.shape[0] * 100 / movies_and_tags_df.shape[0])
percentage_of_movies_with_tags.compute()

72.46687919516845

### JUSTIFICATIVA PARA REMOÇÃO DOS FILMES SEM GÊNERO E TAGS DO PROCESSO DE RECOMENDAÇÃO
Como este sistema fará uso de gêneros e tags para realizar a recomendação dos filmes, obras com estes dois campos vazios não poderão ser incluídas no processo.

In [12]:
movies_and_tags_df["genres"]=movies_and_tags_df["genres"].replace("(no genres listed)", "")
movies_and_tags_df.compute()
movies_with_genres_or_tags = movies_and_tags_df[
    (
        movies_and_tags_df['genres']!=""
    )
    |
    (
        ~movies_and_tags_df['tag'].isna()
    )
]
movies_with_genres_or_tags.compute()

,movieId,title,genres,tag
0,1,Toy Story (1995),adventure|animation|children|comedy|fantasy,owned|imdb top 250|pixar|time travel|children|...
1,2,Jumanji (1995),adventure|children|fantasy,robin williams|time travel|fantasy|based on ch...
2,3,Grumpier Old Men (1995),comedy|romance,funny|best friend|duringcreditsstinger|fishing...
3,4,Waiting to Exhale (1995),comedy|drama|romance,based on novel or book|chick flick|divorce|int...
4,5,Father of the Bride Part II (1995),comedy,aging|baby|confidence|contraception|daughter|g...
...,...,...,...,...
62417,209155,Santosh Subramaniam (2008),action|comedy|romance,NaN
62418,209157,We (2018),drama,NaN
62419,209159,Window of the Soul (2001),documentary,NaN
62420,209163,Bad Poems (2018),comedy|drama,NaN


## Utilizar **genres** e **tags** na definição dos valores de term frequency - tf e inverse document frequency - idf
Basicamente, este trecho é responsável por definir a relevância dos termos utilizados na categorização dos filmes a serem recomendados com base na quantidade de vezes em que as categorias aparecem no documento.

In [13]:
features = ['genres','tag']

def combine_fields(row):
    return f'{row["genres"]}|{row["tag"]}'

def clear_nan(dataframe_with_nan, features): 
    for feature in features:
        dataframe_with_nan[feature] = dataframe_with_nan[feature].fillna('')

clear_nan(movies_with_genres_or_tags, features)
movies_with_genres_or_tags["combinedFeatures"] = movies_with_genres_or_tags.apply(combine_fields, axis=1)
tf_idf_matrix = TfidfVectorizer().fit_transform(movies_with_genres_or_tags["combinedFeatures"])

/Users/jeffersonsantos/movies_recommendation_tcc/.venv/lib/python3.6/site-packages/dask/dataframe/core.py:4325: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=(None, 'object'))

  warnings.warn(meta_warning(meta))


## Dividir a base em 20 partes

Optou-se por este passo a fim de evitar que houvesse qualquer tipo de problema ao rodar este algoritmo em uma máquina local ou até mesmo no Google Collab.

In [14]:
chunk_limit=round(tf_idf_matrix.shape[0]*1/100)

## Gerar a similaridade do cosseno para cada um dos filmes
Construir lista contendo os 20 filmes mais próximos utilizando o espaço vetorial.

In [28]:
movie_recommendation_list = []
movies_data = movies_with_genres_or_tags.compute()
start_chunk_position=0
movie_position=0

def filter_movie_by_id(movie_id, movie_recommended_id):
    return movie_id != movie_recommended_id

def get_movie_recommendation(position_list):
    return {
        "id": int(movies_data["movieId"].iloc[position_list]), 
        "title":  movies_data["title"].iloc[position_list]
    }

def get_movie_with_recommendations(movie_id, recommendation_ids):
    return {
        "movieId": movie_id,
        "moviesRecommendation": recommendation_ids
    }

def get_movie_recommendations(cosine_similarities, movie_position):
    total_movies_recommended = -21
    for recommendation in cosine_similarities:
        movie_id = movies_data["movieId"].iloc[movie_position]
        
        recommendation_positions = recommendation.argsort()[:total_movies_recommended:-1]
        
        recommendation_ids = [get_movie_recommendation(rec_position) for rec_position in recommendation_positions 
                              if filter_movie_by_id(movie_id, movies_data["movieId"].iloc[rec_position])]
        
        movie_recommendation = get_movie_with_recommendations(movie_id, recommendation_ids)
        
        movie_recommendation_list.append(movie_recommendation)
        
        movie_position+=1
        
    return movie_position
        

while start_chunk_position < tf_idf_matrix.shape[0]:
    print(start_chunk_position)
    limit_of_chunk = start_chunk_position + chunk_limit
    cosine_similarities = linear_kernel(tf_idf_matrix[start_chunk_position:limit_of_chunk], tf_idf_matrix)
    movie_position = get_movie_recommendations(cosine_similarities, movie_position)
    start_chunk_position = limit_of_chunk

0
2976
5952
8928
11904
14880
17856
20832
23808
26784
29760
32736
35712
38688
41664
44640
47616
50592
53568
56544


## Gerar dataframe com os filmes recomendados

O trecho de código a seguuir cria um dataframe com os filmes a serem recomendados.

In [23]:
recommendation_bag_dataframe = db.from_sequence(movie_recommendation_list)
df_recommendation = recommendation_bag_dataframe.to_dataframe()
df_recommendation.compute()

,movieId,moviesRecommendation
0,1,"[{'id': 3114, 'title': 'Toy Story 2 (1999)'}, ..."
1,2,"[{'id': 134678, 'title': 'The Games Maker (201..."
2,3,"[{'id': 3450, 'title': 'Grumpy Old Men (1993)'..."
3,4,"[{'id': 2415, 'title': 'Violets Are Blue... (1..."
4,5,"[{'id': 155743, 'title': 'My Big Fat Greek Wed..."
...,...,...
9,209155,"[{'id': 191313, 'title': 'Balupu (2013)'}, {'i..."
10,209157,"[{'id': 164063, 'title': 'Antonia (2015)'}, {'..."
11,209159,"[{'id': 175325, 'title': 'Bird on a Wire (1974..."
12,209163,"[{'id': 159600, 'title': 'China and Sex (1994)..."


## Realizar o merge do dataframe de filmes recomendados com o que contém os principais metadados de filmes

In [24]:
movies_and_tags_df = movies_and_tags_df.merge(df_recommendation, on='movieId', how='left')
clear_nan(movies_and_tags_df, ["tag", "moviesRecommendation"])
movies_and_tags_df.compute()

,movieId,title,genres,tag,moviesRecommendation
0,22,Copycat (1995),crime|drama|horror|mystery|thriller,thriller|cowardliness|police brutality|police ...,"[{'id': 73511, 'title': 'Horsemen (2009)'}, {'..."
1,135,Down Periscope (1996),comedy,misfit|submarine|u.s. navy|good comedy|david s...,"[{'id': 123518, 'title': 'Hell Below (1933)'},..."
2,180,Mallrats (1995),comedy|romance,jason lee|kevin smith|view askew|comedy|crude ...,"[{'id': 1639, 'title': 'Chasing Amy (1997)'}, ..."
3,357,Four Weddings and a Funeral (1994),comedy|romance,romance|bride|bridegroom|bridesmaid|clumsy fel...,"[{'id': 6942, 'title': 'Love Actually (2003)'}..."
4,418,Being Human (1993),drama,medieval times|struggle|success|special,"[{'id': 130028, 'title': 'The Diamond Queen (1..."
...,...,...,...,...,...
605,208403,One Hell of a Christmas (2002),action|comedy|crime,fangoria,"[{'id': 208409, 'title': 'Lady of the Lake (19..."
606,208425,Ville-Marie (2015),drama|thriller,,"[{'id': 137052, 'title': 'A Job to Kill For (2..."
607,208427,10 jours en or (2012),comedy|drama,,"[{'id': 159600, 'title': 'China and Sex (1994)..."
608,208821,"Fireworks, Should We See It from the Side or t...",drama|romance,,"[{'id': 190185, 'title': 'Spring Fever (2010)'..."


## Gerar csv do dataframe com os metadados de filmes

In [25]:
movies_and_tags_df.to_csv(f"{path}/movies_with_recommendations.csv", single_file=True)

['/Users/jeffersonsantos/movies_recommendation_tcc/movies_with_recommendations.csv']

## Mongo Bulk
Salvar filmes no MongoDB usando bulk. Optou-se por recomendar 20 filmes para cada obra. Também foram salvos os dados utilizados no processo de recomendação a título de comprovação acadêmica.

In [30]:
mongo_client = MongoClient(f'mongodb://{os.environ["MONGO_INITDB_ROOT_USERNAME"]}'
                           f':{os.environ["MONGO_INITDB_ROOT_PASSWORD"]}'
                           f'@{os.environ["MONGODB_HOST"]}')
movies_db = mongo_client.movies_recommendation
col = movies_db.recommendation
movies_data = movies_and_tags_df.compute()
json_movies_data = movies_data.to_dict(orient='records')
for data in json_movies_data:
    col.replace_one({'_id': data.get('movieId')}, data, upsert=True)

## Trazendo um exemplo de filme do MongoDB
Optou-se por trazer um filme com base em seu id a fim de demonstrar que a obra foi registrada com sucesso.

In [27]:
mydoc = col.find({"movieId": 260})
print(mydoc)
for doc in mydoc[0:5]:
    print(doc)

{'_id': 260, 'movieId': 260, 'title': 'Star Wars: Episode IV - A New Hope (1977)', 'genres': 'action|adventure|sci-fi', 'tag': 'classic|sci-fi|action|adventure|fantasy|space adventure|classic sci-fi|good vs evil|aliens|oldie but goodie|scifi cult|space|cult classic|futuristic|space action|space opera|old movie|epic|harrison ford|science fiction|space epic|action comedy|romance|entertaining|good story|imdb top 250|scifi|darth vader|luke skywalker|the death star|religion|hero\'s journey|george lucas|star wars|epic adventure|birth of great scifi ideas|old fx quality|story driven|bite me|episode what? it\'s cut off, so i don\'t even know what movie it is|action, scifi|quotable|exciting|fun|heroic journey|inventive|amazing|masterpiece|must see|universe|sf,science fiction|incest|james earl jones|jedi|john williams|robots|atmospheric|future|great soundtrack|science fantasy|space travel|special effects|war|robots and androids|family|shooting|a western set in an alien world|nerdy|a fun introduc